In [3]:
import re, json, pathlib, logging, time, argparse, pprint, math 
import datetime as dt
import importlib.resources as rsrc
from itertools import chain
from operator import itemgetter 
#
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import nltk as nltk
from nltk.corpus import stopwords
from whoosh.lang.porter import stem
#from googletrans import Translator
from pygoogletranslation import Translator
import circlify as crcf
import matplotlib.pyplot as plt
import random

import polmap.polmap as plmp
import postprocess.postprocess as pspr
import keyword


##MM imports
#from polmap.polmap import make_directories, preprocess_text, doc2text, SDGrefs_mapper # replaced the keyword processing block

#import fromRtoPython

from docx2python import docx2python
 
#from polmap.polmap import make_directories, preprocess_text, doc2text # replaced the keyword processing block
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [8]:
results_df = pd.read_excel('output/SDGS_61_Green_Deal_Test/output/6-results/mapping_.xlsx', sheet_name='Summary', index_col=0)
results_df.head()

,Goal,Target,Sum_of_keys,Count_of_keys,list_of_keys
Index,,,,,
1,SDG 1,1.0,0,0,NaN
2,SDG 1,1.1,0,0,NaN
3,SDG 1,1.2,0,0,NaN
4,SDG 1,1.3,0,0,NaN
5,SDG 1,1.4,0,0,NaN


In [33]:
goal=results_df.groupby(by=["Goal"]).sum()
goaL=goal.loc[[f'SDG {n}' for n in range(1,18)]]
targets=results_df[results_df.Goal=='SDG 1']
targets.head(15)
for target, count in zip(targets['Target'], targets['Sum_of_keys']):
    print(target, count)

1.0 0
1.1 0
1.2 0
1.3 0
1.4 0
1.5 0
1.a 0
1.b 0


In [ ]:
df = pd.read_excel('output/The_EU_Green_Deal_/output/6-results/results_.xlsx', sheet_name='priorities', index_col=0)

pp_colors={'Human Development':'#FECA4D',
'Growth Jobs':'#FABE13',
'Green Deal':'#73936F',
'Partnership':'#1A6835',
'Digital':'#72ABDC',
'Governance':'#E2809B',
'Digitalisation':'#4D9CD5',
'Migration':'#DE6189'
}

df.rename(columns = {'priority':'name', 'Count':'size'}, inplace = True)
df['goal_color']=df['name'].map(pp_colors)
df.to_dict('index')


In [ ]:
input = 'output/results_green_deal_test.xlsx'

result_df = pd.ExcelFile(input)

print(result_df.sheet_names)

targets, goals, total = result_df.parse('target_count'), result_df.parse('goal_overview'), result_df.parse('total_count_(goals_+_targets)')

# print(result_df.sheet_names)

In [ ]:
targets.rename(columns={'Count':'Target_Count', 'Num_keys':'Tgt_nr.keys'}, inplace=True) 
targets.groupby(['Policy','Goal','Target']).sum()
dat_target_join_keys = raw_df.groupby(['Policy', 'Target'])["Keyword"].apply(lambda x : " - ".join(x.astype(str))).reset_index()

In [ ]:
targets.head

In [ ]:
goals.rename(columns = {'Count':'Goal_Count', 'Goal_nr.keys'}, inplace = True) 
goals.groupby(['Policy','Goal']).sum()